# Tutorial 6: Advanced Usage - Working with SQL

In this tutorial, we will showcase a few tips and tricks that helps you more easily move between Ponder and SQL. We will be using the [MIMIC-III demo dataset](https://physionet.org/content/mimiciii-demo/1.4/) as an example dataset. The MIMIC-III Clinical Database contains deidentified health-related data of patients who stayed in an intensive care unit (ICU) at the Beth Israel Deaconess Medical Center in Boston. The demo dataset contains records for 100 patients across three tables `PATIENTS`, `ICUSTAYS`, and `ADMISSIONS`. 

### Data Definition (DDL) with SQLAlchemy

In SQL, DDL statements involve modifications to the database schema, e.g., `CREATE`, `ALTER`, `DROP`. Oftentimes, you may want to run a DDL statement alongside your analysis, either via an external query editor or through SQLAlchemy. 



In [1]:
# Install SQLAlchemy if you don't have it already
! pip install --upgrade snowflake-sqlalchemy --quiet

In [2]:
import os; os.chdir("..");

In [3]:
import credential

from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
engine = create_engine(URL(
    account = credential.params["account"],
    user = credential.params["user"],
    password = credential.params["password"],
    database = credential.params["database"],
    schema = credential.params["schema"],
    warehouse = credential.params["warehouse"],
    role=credential.params["role"],
))
connection = engine.connect()

Then we can run the SQL query directly to create a new database named `MIMIC3`

In [4]:
connection.execute("CREATE DATABASE IF NOT EXISTS MIMIC3;") 

Now if we print out the database, we can see the new table MIMIC3 added: 

In [5]:
ret = connection.execute("SHOW DATABASES;") 

In [6]:
import pandas
pandas.DataFrame(ret.all())

,created_on,name,is_default,is_current,origin,owner,comment,options,retention_time
0,2023-01-25 16:54:24-08:00,BOOKS,N,N,,ACCOUNTADMIN,,,1
1,2023-02-09 18:29:33.032000-08:00,CUSTOMER,N,N,,ACCOUNTADMIN,,,1
2,2023-03-13 20:19:42.684000-07:00,MIMIC3,N,N,,ACCOUNTADMIN,,,1
3,2023-01-11 17:38:46.515000-08:00,SNOWFLAKE,N,N,SNOWFLAKE.ACCOUNT_USAGE,,,,1
4,2023-01-11 17:38:49.125000-08:00,SNOWFLAKE_SAMPLE_DATA,N,N,SFSALESSHARED.SFC_SAMPLES_PROD3.SAMPLE_DATA,ACCOUNTADMIN,Provided by Snowflake during account provisioning,,1
5,2023-01-17 19:15:35.905000-08:00,TEST,N,Y,,ACCOUNTADMIN,,,1


### Existing SQL DML with Ponder

In [7]:
import os; os.chdir("..")

We will be using a few example tables for the remainder of this tutorial. You can run this python script to populate the required datasets to your database. This will populate three different tables `PATIENTS`, `ADMISSIONS`, and `ICUSTAYS` to your database. 

In [ ]:
!python populate_mimic3.py > /dev/null 2>&1

Oftentime, you may already have an existing SQL script that you've been using to join and denormalize some tables or perform some pre-aggregation or ETL before you perform your analysis. You want to reuse that SQL code while working with Ponder for the remaining analysis workflow. In this example, we show how you can feed this into the `pd.read_sql` to operate on the resulting table.

In [8]:
import credential
import ponder.snowflake
import modin.pandas as pd
credential.params["database"] = "MIMIC3"
snowflake_con = ponder.snowflake.connect(user=credential.params["user"],password=credential.params["password"],account=credential.params["account"],role=credential.params["role"],database=credential.params["database"],schema=credential.params["schema"],warehouse=credential.params["warehouse"])
ponder.snowflake.init(snowflake_con)

For example, we may want to use [this existing SQL query](https://mimic.mit.edu/docs/iii/tutorials/intro-to-mimic-iii/#5-patient-age-and-mortality) from the MIT MIMIC-III tutorial to jumpstart our analysis. 

In [9]:
df = pd.read_sql('''SELECT p."subject_id", p."dob", a."hadm_id",
                    a."admittime", p."expire_flag"
                    FROM MIMIC3.PUBLIC.ADMISSIONS as a
                    INNER JOIN MIMIC3.PUBLIC.PATIENTS as p
                    ON p."subject_id" = a."subject_id"''', con = snowflake_con)

In [10]:
df

,subject_id,dob,hadm_id,admittime,expire_flag
0,10006,2094-03-05 00:00:00,142345,2164-10-23 21:09:00,1
1,10011,2090-06-05 00:00:00,105331,2126-08-14 22:32:00,1
2,10013,2038-09-03 00:00:00,165520,2125-10-04 23:36:00,1
3,10017,2075-09-21 00:00:00,199207,2149-05-26 17:19:00,1
4,10019,2114-06-20 00:00:00,177759,2163-05-14 20:43:00,1
...,...,...,...,...,...
124,44083,2057-11-15 00:00:00,198330,2112-05-28 15:45:00,1
125,44154,1878-05-14 00:00:00,174245,2178-05-14 20:29:00,1
126,44212,2078-06-16 00:00:00,163189,2123-11-24 14:14:00,1
127,44222,2107-06-27 00:00:00,192189,2180-07-19 06:55:00,1


Then we can continue using Ponder by writing pandas as always.

In [11]:
df["dob"] = pd.to_datetime(df["dob"])
df["admittime"] = pd.to_datetime(df["admittime"])

In [12]:
df["age"] = df["admittime"].dt.year  - df["dob"].dt.year

In [13]:
df["age"]

0       70
1       36
2       87
3       74
4       49
      ... 
124     55
125    300
126     45
127     73
128     58
Name: age, Length: 129, dtype: int32

### Working with multiple tables

With Ponder, you can work with multiple tables at the same time by creating different dataframes using the `read_sql` or `read_csv` command.

In [14]:
patients = pd.read_sql("PATIENTS", con=snowflake_con)
admissions = pd.read_sql("ADMISSIONS", con=snowflake_con)

Now we can work with these two dataframes in pandas. Here, we perform the same query as the SQL query above: 
```sql
SELECT p."subject_id", p."dob", a."hadm_id",
        a."admittime", p."expire_flag"
        FROM MIMIC3.PUBLIC.ADMISSIONS as a
        INNER JOIN MIMIC3.PUBLIC.PATIENTS as p
        ON p."subject_id" = a."subject_id"
```

In [16]:
patients.merge(admissions,on="subject_id")[["subject_id", "dob", "hadm_id","admittime", "expire_flag"]]

,subject_id,dob,hadm_id,admittime,expire_flag
0,10006,2094-03-05 00:00:00,142345,2164-10-23 21:09:00,1
1,10011,2090-06-05 00:00:00,105331,2126-08-14 22:32:00,1
2,10013,2038-09-03 00:00:00,165520,2125-10-04 23:36:00,1
3,10017,2075-09-21 00:00:00,199207,2149-05-26 17:19:00,1
4,10019,2114-06-20 00:00:00,177759,2163-05-14 20:43:00,1
...,...,...,...,...,...
124,44083,2057-11-15 00:00:00,198330,2112-05-28 15:45:00,1
125,44154,1878-05-14 00:00:00,174245,2178-05-14 20:29:00,1
126,44212,2078-06-16 00:00:00,163189,2123-11-24 14:14:00,1
127,44222,2107-06-27 00:00:00,192189,2180-07-19 06:55:00,1
